# RL GPU Scheduling - FINAL COMPARISON
## Comparing Original Paper, Original Colab, and Our Improvements

---

## Summary of Approaches

### 1. Original Paper/Colab Configuration
- **Environment**: Pandas-based (slow)
- **Deadlines**: 1.0-1.5× fastest completion (very tight)
- **Network**: [256, 256]
- **Training**: 200k timesteps
- **Baselines**: None implemented in colab
- **Result**: ~85-90% late jobs (too tight deadlines make problem trivial)

### 2. Our Optimizations Tried

| Version | Change | Result | Issue |
|---------|--------|--------|-------|
| **Fast** | NumPy env (10-50× faster) | Same accuracy, faster | Training/eval mismatch |
| **Improved** | +LR annealing, +entropy decay, deeper net | ~48% late | Still worse than heuristics |
| **Relaxed** | Deadlines 2-4× | ~48% late | RL still losing |
| **Enhanced** | +Slice sizes in obs, +immediate rewards | Pending | Better signal for learning |

### 3. Key Finding

**The original problem formulation is greedy-optimal:**
- Larger slice → faster completion → less tardiness
- "Largest-First" heuristic achieves near-optimal results
- RL struggles to discover this simple rule from scratch

---

## This Notebook

Runs **fair comparison** of:
1. **Original Config** - Paper's hyperparameters, tight deadlines
2. **Improved Config** - Our enhancements, tight deadlines
3. **Relaxed Config** - Our enhancements, relaxed deadlines
4. **Heuristic Baselines** - EFT, Largest-First, Smallest-First, Random

In [ ]:
# Cell 1: Install
%pip install -q stable-baselines3 sb3-contrib gymnasium

In [ ]:
# Cell 2: Imports
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

import torch
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from sb3_contrib.ppo_mask import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.utils import get_action_masks

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
if DEVICE == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    print('⚠️  WARNING: Using CPU - training will be slow!')
    print('   Enable GPU runtime: Runtime → Change runtime type → GPU')

In [ ]:
# Cell 3: Constants (same as original paper)
MIG_PROFILE = {
    1: [(7, 40)], 2: [(4, 20), (3, 20)], 3: [(4, 20), (2, 10), (1, 10)],
    4: [(4, 20), (1, 5), (1, 5), (1, 5)], 5: [(3, 20), (3, 20)],
    6: [(3, 20), (2, 10), (1, 10)], 7: [(3, 20), (1, 10), (1, 5), (1, 5)],
    8: [(2, 10), (2, 10), (3, 20)], 9: [(2, 10), (1, 5), (1, 5), (3, 20)],
    10: [(1, 5), (1, 5), (2, 10), (3, 20)], 11: [(1, 5), (1, 5), (1, 5), (1, 5), (3, 20)],
    12: [(2, 10), (2, 10), (2, 10), (1, 10)], 13: [(2, 10), (1, 5), (1, 5), (2, 10), (1, 10)],
    14: [(1, 5), (1, 5), (2, 10), (2, 10), (1, 10)], 15: [(2, 10), (1, 10), (1, 5), (1, 5), (1, 5), (1, 5)],
    16: [(1, 5), (1, 5), (2, 10), (1, 10), (1, 5), (1, 5)],
    17: [(1, 5), (1, 5), (1, 10), (1, 5), (2, 10), (1, 5)],
    18: [(1, 5), (1, 5), (1, 10), (1, 5), (1, 5), (2, 10)],
    19: [(1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]
}

ENERGY_TABLE = np.array([40, 120, 160, 200, 240, 250, 250, 250], dtype=np.float32)
SLICE_DUR_IDX = {1: 2, 2: 3, 3: 4, 4: 5, 7: 6}
INTERARRIVALS = np.array([0.111, 0.083, 0.085, 0.1, 0.137, 0.169, 0.171, 0.169, 0.179, 0.191,
    0.201, 0.188, 0.17, 0.177, 0.168, 0.171, 0.163, 0.138, 0.12, 0.111,
    0.129, 0.116, 0.106, 0.104, 0.111], dtype=np.float32)

GPU_CONFIG = [1, 1, 2, 2, 3, 3, 12, 12]
TIME_SCALE = 100.0
MAX_QUEUE_SIZE = 100

print("Constants loaded (matching original paper)")

In [ ]:
# Cell 4: Queue generation - supports both tight and relaxed deadlines
def create_queue(hour_range=24, deadline_min=1.0, deadline_max=1.5, seed=None):
    """
    Create job queue.
    
    Args:
        deadline_min, deadline_max: Multipliers for fastest completion time
            Original paper: 1.0-1.5 (very tight)
            Relaxed: 2.0-4.0 (learnable)
    """
    if seed is not None:
        np.random.seed(seed)
    
    jobs = []
    arrival = 0.0
    max_time = hour_range * 60.0
    
    while arrival < max_time:
        hour_idx = min(int(arrival / 60), 24)
        rate = INTERARRIVALS[hour_idx] * 20
        arrival += np.random.exponential(1.0 / rate)
        if arrival >= max_time:
            break
        
        is_inference = np.random.random() < 0.8
        if is_inference:
            g1 = np.random.exponential(3.0)
            if np.random.randint(3) == 2:
                g2, g3, g4, g7 = g1/2, g1/3, g1/12.5*3.2, g1/18.4*3.2
            else:
                g2, g3, g4, g7 = g1/2, g1/3, g1/4, g1/7
        else:
            g1 = np.random.lognormal((np.log(40)+np.log(60))/2, (np.log(60)-np.log(40))/3.29)
            if np.random.randint(3) == 2:
                g2, g3, g4, g7 = g1/6*3.4, g1/7.85*3.4, g1/8.4*3.4, g1/9.75*3.4
            else:
                g2, g3, g4, g7 = g1/4.1*2.2, g1/5.8*2.2, g1/7.1*2.2, g1/10.5*2.2
        
        deadline = arrival + np.random.uniform(deadline_min, deadline_max) * g7
        jobs.append([arrival, deadline, g1, g2, g3, g4, g7])
    
    return np.array(jobs, dtype=np.float32)

# Test both
q_tight = create_queue(24, 1.0, 1.5, seed=42)
q_relaxed = create_queue(24, 2.0, 4.0, seed=42)
slack_tight = np.mean((q_tight[:, 1] - q_tight[:, 0]) / q_tight[:, 6])
slack_relaxed = np.mean((q_relaxed[:, 1] - q_relaxed[:, 0]) / q_relaxed[:, 6])

print(f"Queue sizes: {len(q_tight)} jobs")
print(f"Tight deadlines (original): {slack_tight:.2f}× avg slack")
print(f"Relaxed deadlines (ours):   {slack_relaxed:.2f}× avg slack")

In [ ]:
# Cell 5: Environment (NumPy-optimized, supports both configurations)
class SchedulingEnv(gym.Env):
    """NumPy-optimized scheduling environment."""
    
    def __init__(self, gpu_config, queue=None, hour_range=24, 
                 deadline_min=1.0, deadline_max=1.5, enhanced_obs=False):
        super().__init__()
        self.gpu_config = gpu_config
        self.hour_range = hour_range
        self.external_queue = queue
        self.deadline_min = deadline_min
        self.deadline_max = deadline_max
        self.enhanced_obs = enhanced_obs
        
        slices = []
        for gpu_id, cfg in enumerate(gpu_config):
            for size, _ in MIG_PROFILE[cfg]:
                slices.append((gpu_id, len(slices), size))
        self.slice_info = np.array(slices, dtype=np.int32)
        self.n_slices = len(slices)
        self.n_gpus = len(gpu_config)
        self.slice_sizes_norm = self.slice_info[:, 2].astype(np.float32) / 7.0
        
        # Observation space
        if enhanced_obs:
            self.observation_space = spaces.Dict({
                "next_job": spaces.Box(-np.inf, np.inf, shape=(5,), dtype=np.float32),
                "queue_stats": spaces.Box(0, 1, shape=(40,), dtype=np.float32),
                "slice_busy": spaces.Box(0, 1, shape=(self.n_slices,), dtype=np.float32),
                "slice_sizes": spaces.Box(0, 1, shape=(self.n_slices,), dtype=np.float32),
                "extras": spaces.Box(0, 1, shape=(3,), dtype=np.float32),
            })
            self._obs_next_job = np.zeros(5, dtype=np.float32)
            self._obs_extras = np.zeros(3, dtype=np.float32)
        else:
            self.observation_space = spaces.Dict({
                "next_job": spaces.Box(-np.inf, np.inf, shape=(4,), dtype=np.float32),
                "queue_stats": spaces.Box(0, 1, shape=(40,), dtype=np.float32),
                "slices": spaces.Box(0, 1, shape=(self.n_slices,), dtype=np.float32),
                "extras": spaces.Box(0, 1, shape=(2,), dtype=np.float32),
            })
            self._obs_next_job = np.zeros(4, dtype=np.float32)
            self._obs_extras = np.zeros(2, dtype=np.float32)
        
        self.action_space = spaces.Discrete(self.n_slices)
        self._obs_queue_stats = np.zeros(40, dtype=np.float32)
        self._bins = np.array([-100, 0, 0.05, 0.2, 0.5, 1, 5, 10, 20, 30, 1e9], dtype=np.float32)
    
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        if self.external_queue is not None:
            self.jobs = self.external_queue.copy()
        else:
            self.jobs = create_queue(self.hour_range, self.deadline_min, self.deadline_max, seed)
        self.n_jobs = len(self.jobs)
        self.slice_busy = np.zeros(self.n_slices, dtype=np.int32)
        self.slice_job = np.full(self.n_slices, -1, dtype=np.int32)
        self.slice_finish = np.zeros(self.n_slices, dtype=np.float32)
        self.gpu_energy_time = np.zeros(self.n_gpus, dtype=np.float32)
        self.now = 0.0
        self.next_arrival_idx = 0
        self.working_queue = []
        self.completed = np.zeros(self.n_jobs, dtype=bool)
        self.total_tardiness = 0.0
        self.total_energy = 0.0
        self.num_late = 0
        
        if self.n_jobs > 0:
            self.now = self.jobs[0, 0]
            self.working_queue.append(0)
            self.next_arrival_idx = 1
        return self._get_obs(), {}
    
    def _get_obs(self):
        if self.working_queue:
            self.working_queue.sort(key=lambda j: self.jobs[j, 1])
            j = self.working_queue[0]
            time_to_deadline = self.jobs[j, 1] - self.now
            self._obs_next_job[0] = time_to_deadline / TIME_SCALE
            self._obs_next_job[1] = (self.jobs[j, 2] + self.jobs[j, 3]) / 2 / TIME_SCALE
            self._obs_next_job[2] = (self.jobs[j, 4] + self.jobs[j, 5]) / 2 / TIME_SCALE
            self._obs_next_job[3] = self.jobs[j, 6] / TIME_SCALE
            if self.enhanced_obs:
                self._obs_next_job[4] = min(1.0, self.jobs[j, 6] / max(time_to_deadline, 0.01))
            
            wq = np.array(self.working_queue)
            n = len(wq)
            self._obs_queue_stats[0:10] = np.histogram(self.jobs[wq, 1] - self.now, self._bins)[0] / n
            self._obs_queue_stats[10:20] = np.histogram((self.jobs[wq, 2] + self.jobs[wq, 3]) / 2, self._bins)[0] / n
            self._obs_queue_stats[20:30] = np.histogram((self.jobs[wq, 4] + self.jobs[wq, 5]) / 2, self._bins)[0] / n
            self._obs_queue_stats[30:40] = np.histogram(self.jobs[wq, 6], self._bins)[0] / n
        else:
            self._obs_next_job.fill(0)
            self._obs_queue_stats.fill(0)
        
        free_mask = self.slice_busy == 0
        n_free = np.sum(free_mask)
        
        if self.enhanced_obs:
            self._obs_extras[0] = min(len(self.working_queue) / MAX_QUEUE_SIZE, 1.0)
            self._obs_extras[1] = n_free / self.n_slices
            self._obs_extras[2] = np.max(self.slice_info[free_mask, 2]) / 7.0 if n_free > 0 else 0.0
            return {
                "next_job": self._obs_next_job.copy(),
                "queue_stats": self._obs_queue_stats.copy(),
                "slice_busy": self.slice_busy.astype(np.float32),
                "slice_sizes": self.slice_sizes_norm.copy(),
                "extras": self._obs_extras.copy(),
            }
        else:
            self._obs_extras[0] = min(len(self.working_queue) / MAX_QUEUE_SIZE, 1.0)
            self._obs_extras[1] = n_free / self.n_slices
            return {
                "next_job": self._obs_next_job.copy(),
                "queue_stats": self._obs_queue_stats.copy(),
                "slices": self.slice_busy.astype(np.float32),
                "extras": self._obs_extras.copy(),
            }
    
    def valid_action_mask(self):
        return self.slice_busy == 0
    
    def _calc_energy(self, gpu_id):
        mask = self.slice_info[:, 0] == gpu_id
        busy_sizes = self.slice_info[mask & (self.slice_busy == 1), 2]
        util = min(int(np.sum(busy_sizes)), 7)
        energy = ENERGY_TABLE[util] * (self.now - self.gpu_energy_time[gpu_id])
        self.total_energy += energy
        self.gpu_energy_time[gpu_id] = self.now
    
    def step(self, action):
        job_idx = self.working_queue.pop(0)
        slice_size = self.slice_info[action, 2]
        gpu_id = self.slice_info[action, 0]
        duration = self.jobs[job_idx, SLICE_DUR_IDX[slice_size]]
        
        self._calc_energy(gpu_id)
        self.slice_busy[action] = 1
        self.slice_job[action] = job_idx
        self.slice_finish[action] = self.now + duration
        
        if self.working_queue and np.any(self.slice_busy == 0):
            return self._get_obs(), 0.0, False, False, {'action_mask': self.valid_action_mask()}
        
        step_tardiness = 0.0
        num_completions = 0
        
        while True:
            next_arrival = self.jobs[self.next_arrival_idx, 0] if self.next_arrival_idx < self.n_jobs else 1e12
            busy_mask = self.slice_busy == 1
            next_completion = np.min(self.slice_finish[busy_mask]) if np.any(busy_mask) else 1e12
            
            if next_arrival >= 1e12 and next_completion >= 1e12:
                break
            
            if next_arrival <= next_completion:
                self.now = next_arrival
                self.working_queue.append(self.next_arrival_idx)
                self.next_arrival_idx += 1
            else:
                self.now = next_completion
                completing = np.where((self.slice_finish <= self.now + 1e-9) & busy_mask)[0]
                for s in completing:
                    j = self.slice_job[s]
                    tardiness = max(0.0, self.now - self.jobs[j, 1])
                    if tardiness > 0:
                        self.total_tardiness += tardiness
                        step_tardiness += tardiness
                        self.num_late += 1
                    self.completed[j] = True
                    self.slice_busy[s] = 0
                    self.slice_job[s] = -1
                    num_completions += 1
            
            for g in range(self.n_gpus):
                self._calc_energy(g)
            
            if self.working_queue and np.any(self.slice_busy == 0):
                break
            if self.next_arrival_idx >= self.n_jobs and not np.any(self.slice_busy == 1) and not self.working_queue:
                break
        
        terminated = np.all(self.completed)
        
        if terminated:
            reward = (-self.total_tardiness - 0.0000225 * self.total_energy) / (self.n_jobs * 0.0000225 + 1)
            info = {
                'total_energy': self.total_energy,
                'avg_tardiness': self.total_tardiness / self.n_jobs,
                'num_late_jobs': self.num_late,
                'total_jobs': self.n_jobs,
            }
        else:
            reward = (-step_tardiness - 0.0000225 * self.total_energy) / (max(1, num_completions) * 1.0000225)
            info = {'total_energy': self.total_energy}
        
        info['action_mask'] = self.valid_action_mask()
        return self._get_obs(), reward, terminated, False, info

print("Environment ready (supports both original and improved configs)")

In [ ]:
# Cell 6: Callbacks
class ProgressCallback(BaseCallback):
    def __init__(self, check_freq=10000):
        super().__init__()
        self.check_freq = check_freq
        self.start_time = None
    def _on_training_start(self):
        self.start_time = time.time()
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            elapsed = time.time() - self.start_time
            sps = self.n_calls / elapsed
            remaining = (self.model._total_timesteps - self.n_calls) / sps / 60
            print(f"  Step {self.n_calls:,}: {sps:.0f} sps, ~{remaining:.1f} min left")
        return True

class LRScheduleCallback(BaseCallback):
    def __init__(self, initial=3e-4, final=1e-5):
        super().__init__()
        self.initial = initial
        self.final = final
    def _on_step(self):
        progress = self.num_timesteps / self.model._total_timesteps
        lr = self.initial + progress * (self.final - self.initial)
        for pg in self.model.policy.optimizer.param_groups:
            pg['lr'] = lr
        return True

class EntropyDecayCallback(BaseCallback):
    def __init__(self, initial=0.01, final=0.001):
        super().__init__()
        self.initial = initial
        self.final = final
    def _on_step(self):
        progress = self.num_timesteps / self.model._total_timesteps
        self.model.ent_coef = self.initial + progress * (self.final - self.initial)
        return True

print("Callbacks ready")

In [ ]:
# Cell 7: Training function
def mask_fn(env):
    return env.valid_action_mask()

def train_model(config_name, deadline_min, deadline_max, enhanced_obs, 
                net_arch, batch_size, n_epochs, timesteps, use_annealing):
    """Train a model with specified configuration."""
    
    print(f"\n{'='*60}")
    print(f"TRAINING: {config_name}")
    print(f"{'='*60}")
    print(f"  Deadlines: {deadline_min}-{deadline_max}×")
    print(f"  Network: {net_arch}")
    print(f"  Batch: {batch_size}, Epochs: {n_epochs}")
    print(f"  Timesteps: {timesteps:,}")
    print(f"  LR/Entropy annealing: {use_annealing}")
    print(f"  Enhanced obs: {enhanced_obs}")
    
    def make_env():
        def _init():
            env = SchedulingEnv(GPU_CONFIG, hour_range=24, 
                               deadline_min=deadline_min, deadline_max=deadline_max,
                               enhanced_obs=enhanced_obs)
            return ActionMasker(env, mask_fn)
        return _init
    
    train_env = DummyVecEnv([make_env() for _ in range(4)])
    
    model = MaskablePPO(
        "MultiInputPolicy",
        train_env,
        verbose=0,
        device=DEVICE,
        n_steps=1024,
        batch_size=batch_size,
        n_epochs=n_epochs,
        learning_rate=3e-4,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2 if not use_annealing else 0.15,
        ent_coef=0.001 if not use_annealing else 0.01,
        vf_coef=0.5,
        max_grad_norm=0.5,
        policy_kwargs=dict(net_arch=net_arch),
    )
    
    if use_annealing:
        callbacks = CallbackList([
            ProgressCallback(check_freq=20000),
            LRScheduleCallback(3e-4, 1e-5),
            EntropyDecayCallback(0.01, 0.001),
        ])
    else:
        callbacks = ProgressCallback(check_freq=20000)
    
    t0 = time.time()
    model.learn(total_timesteps=timesteps, callback=callbacks)
    elapsed = time.time() - t0
    print(f"  ✓ Completed in {elapsed/60:.1f} min")
    
    train_env.close()
    return model

print("Training function ready")

In [ ]:
# Cell 8: Evaluation function
def evaluate_model(model, config_name, deadline_min, deadline_max, enhanced_obs, n_episodes=10):
    """Evaluate model and return results."""
    
    results = {'tardiness': [], 'late_frac': [], 'energy': []}
    
    np.random.seed(42)
    seeds = [np.random.randint(0, 100000) for _ in range(n_episodes)]
    
    for seed in seeds:
        np.random.seed(seed)
        env = ActionMasker(SchedulingEnv(GPU_CONFIG, hour_range=24,
                                         deadline_min=deadline_min, deadline_max=deadline_max,
                                         enhanced_obs=enhanced_obs), mask_fn)
        obs, _ = env.reset()
        done = False
        while not done:
            mask = get_action_masks(env)
            action, _ = model.predict(obs, action_masks=mask, deterministic=True)
            obs, _, terminated, truncated, info = env.step(action)
            done = terminated or truncated
        results['tardiness'].append(info['avg_tardiness'])
        results['late_frac'].append(info['num_late_jobs'] / info['total_jobs'])
        results['energy'].append(info['total_energy'])
    
    return results

def evaluate_heuristics(deadline_min, deadline_max, n_episodes=10):
    """Evaluate heuristic baselines."""
    
    def largest_first(mask, env):
        valid = np.where(mask)[0]
        sizes = [env.unwrapped.slice_info[a, 2] for a in valid]
        return valid[np.argmax(sizes)]
    
    def smallest_first(mask, env):
        valid = np.where(mask)[0]
        sizes = [env.unwrapped.slice_info[a, 2] for a in valid]
        return valid[np.argmin(sizes)]
    
    def random_policy(mask, env):
        return np.random.choice(np.where(mask)[0])
    
    def eft_policy(mask, env, obs):
        valid = np.where(mask)[0]
        best_action = valid[0]
        best_finish = float('inf')
        for a in valid:
            size = env.unwrapped.slice_info[a, 2]
            dur = obs['next_job'][3] * TIME_SCALE * (7.0 / size)  # Approximate
            if dur < best_finish:
                best_finish = dur
                best_action = a
        return best_action
    
    heuristics = {
        'Largest-First': lambda m, e, o: largest_first(m, e),
        'Smallest-First': lambda m, e, o: smallest_first(m, e),
        'EFT': eft_policy,
        'Random': lambda m, e, o: random_policy(m, e),
    }
    
    all_results = {}
    
    np.random.seed(42)
    seeds = [np.random.randint(0, 100000) for _ in range(n_episodes)]
    
    for name, policy in heuristics.items():
        results = {'tardiness': [], 'late_frac': [], 'energy': []}
        for seed in seeds:
            np.random.seed(seed)
            env = ActionMasker(SchedulingEnv(GPU_CONFIG, hour_range=24,
                                             deadline_min=deadline_min, deadline_max=deadline_max), mask_fn)
            obs, _ = env.reset()
            done = False
            while not done:
                mask = get_action_masks(env)
                action = policy(mask, env, obs)
                obs, _, terminated, truncated, info = env.step(action)
                done = terminated or truncated
            results['tardiness'].append(info['avg_tardiness'])
            results['late_frac'].append(info['num_late_jobs'] / info['total_jobs'])
            results['energy'].append(info['total_energy'])
        all_results[name] = results
    
    return all_results

print("Evaluation functions ready")

In [ ]:
# Cell 9: TRAIN ALL CONFIGURATIONS
print("="*70)
print("TRAINING ALL CONFIGURATIONS")
print("="*70)
print("\nThis will train 3 models:")
print("  1. Original Paper Config (tight deadlines, basic hyperparams)")
print("  2. Improved Config (tight deadlines, our improvements)")
print("  3. Relaxed Config (relaxed deadlines, our improvements)")
print("\nEstimated time: ~45-90 min total on A100")

# 1. Original Paper Config
model_original = train_model(
    config_name="Original Paper",
    deadline_min=1.0, deadline_max=1.5,  # Tight (original)
    enhanced_obs=False,
    net_arch=[256, 256],  # Original
    batch_size=2048,      # Original
    n_epochs=5,           # Original
    timesteps=200_000,    # Original
    use_annealing=False,  # Original
)

# 2. Improved Config (tight deadlines)
model_improved = train_model(
    config_name="Improved (Tight)",
    deadline_min=1.0, deadline_max=1.5,  # Tight (same as original)
    enhanced_obs=False,
    net_arch=[256, 256, 128],  # Deeper
    batch_size=4096,           # Larger
    n_epochs=8,                # More
    timesteps=200_000,         # Same
    use_annealing=True,        # NEW
)

# 3. Relaxed Config
model_relaxed = train_model(
    config_name="Improved (Relaxed)",
    deadline_min=2.0, deadline_max=4.0,  # RELAXED
    enhanced_obs=True,                    # Enhanced
    net_arch=[256, 256, 128],
    batch_size=4096,
    n_epochs=8,
    timesteps=200_000,
    use_annealing=True,
)

print("\n" + "="*70)
print("ALL MODELS TRAINED!")
print("="*70)

In [ ]:
# Cell 10: EVALUATE ALL
print("="*70)
print("EVALUATING ALL CONFIGURATIONS")
print("="*70)

N_EVAL = 10

# Evaluate RL models
print("\nEvaluating RL models...")
results_original = evaluate_model(model_original, "Original", 1.0, 1.5, False, N_EVAL)
print(f"  Original Paper: {np.mean(results_original['late_frac'])*100:.1f}% late")

results_improved = evaluate_model(model_improved, "Improved", 1.0, 1.5, False, N_EVAL)
print(f"  Improved (Tight): {np.mean(results_improved['late_frac'])*100:.1f}% late")

results_relaxed = evaluate_model(model_relaxed, "Relaxed", 2.0, 4.0, True, N_EVAL)
print(f"  Improved (Relaxed): {np.mean(results_relaxed['late_frac'])*100:.1f}% late")

# Evaluate heuristics on TIGHT deadlines
print("\nEvaluating heuristics (tight deadlines)...")
heuristics_tight = evaluate_heuristics(1.0, 1.5, N_EVAL)
for name, res in heuristics_tight.items():
    print(f"  {name}: {np.mean(res['late_frac'])*100:.1f}% late")

# Evaluate heuristics on RELAXED deadlines
print("\nEvaluating heuristics (relaxed deadlines)...")
heuristics_relaxed = evaluate_heuristics(2.0, 4.0, N_EVAL)
for name, res in heuristics_relaxed.items():
    print(f"  {name}: {np.mean(res['late_frac'])*100:.1f}% late")

print("\n✓ Evaluation complete!")

In [ ]:
# Cell 11: COMPREHENSIVE RESULTS TABLE
print("\n" + "="*90)
print("COMPREHENSIVE RESULTS COMPARISON")
print("="*90)

def fmt_result(results):
    late = np.mean(results['late_frac']) * 100
    late_std = np.std(results['late_frac']) * 100
    tard = np.mean(results['tardiness'])
    tard_std = np.std(results['tardiness'])
    energy = np.mean(results['energy']) / 1e6
    return f"{late:5.1f}±{late_std:4.1f}%  {tard:6.2f}±{tard_std:5.2f}  {energy:5.2f}"

print(f"\n{'Method':<35} {'Late %':>12} {'Tardiness':>14} {'Energy(MJ)':>10}")
print("-"*85)

print("\n--- TIGHT DEADLINES (Original Paper Config: 1.0-1.5×) ---")
print(f"{'RL: Original Paper Config':<35} {fmt_result(results_original)}")
print(f"{'RL: Our Improvements (Tight)':<35} {fmt_result(results_improved)}")
for name, res in heuristics_tight.items():
    print(f"{name:<35} {fmt_result(res)}")

print("\n--- RELAXED DEADLINES (Our Config: 2.0-4.0×) ---")
print(f"{'RL: Our Improvements (Relaxed)':<35} {fmt_result(results_relaxed)}")
for name, res in heuristics_relaxed.items():
    print(f"{name:<35} {fmt_result(res)}")

print("\n" + "="*90)

In [ ]:
# Cell 12: ANALYSIS
print("\n" + "="*90)
print("ANALYSIS")
print("="*90)

# Tight deadlines analysis
rl_orig_late = np.mean(results_original['late_frac'])
rl_impr_late = np.mean(results_improved['late_frac'])
best_heur_tight = min(heuristics_tight.items(), key=lambda x: np.mean(x[1]['late_frac']))
best_heur_tight_late = np.mean(best_heur_tight[1]['late_frac'])

print("\n1. TIGHT DEADLINES (Original Paper Setup)")
print(f"   Best heuristic: {best_heur_tight[0]} ({best_heur_tight_late*100:.1f}% late)")
print(f"   RL Original:    {rl_orig_late*100:.1f}% late")
print(f"   RL Improved:    {rl_impr_late*100:.1f}% late")

if rl_impr_late < best_heur_tight_late:
    imp = (best_heur_tight_late - rl_impr_late) / best_heur_tight_late * 100
    print(f"   → RL WINS by {imp:.1f}%")
else:
    gap = (rl_impr_late - best_heur_tight_late) / best_heur_tight_late * 100
    print(f"   → RL loses by {gap:.1f}% (tight deadlines are greedy-optimal)")

# Relaxed deadlines analysis
rl_relax_late = np.mean(results_relaxed['late_frac'])
best_heur_relax = min(heuristics_relaxed.items(), key=lambda x: np.mean(x[1]['late_frac']))
best_heur_relax_late = np.mean(best_heur_relax[1]['late_frac'])

print("\n2. RELAXED DEADLINES (Our Setup)")
print(f"   Best heuristic: {best_heur_relax[0]} ({best_heur_relax_late*100:.1f}% late)")
print(f"   RL Improved:    {rl_relax_late*100:.1f}% late")

if rl_relax_late < best_heur_relax_late:
    imp = (best_heur_relax_late - rl_relax_late) / best_heur_relax_late * 100
    print(f"   → RL WINS by {imp:.1f}%")
else:
    gap = (rl_relax_late - best_heur_relax_late) / best_heur_relax_late * 100
    print(f"   → RL loses by {gap:.1f}%")

print("\n" + "="*90)

In [ ]:
# Cell 13: VISUALIZATION
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Tight Deadlines
ax = axes[0]
methods_tight = ['RL (Original)', 'RL (Improved)'] + list(heuristics_tight.keys())
lates_tight = [np.mean(results_original['late_frac'])*100, 
               np.mean(results_improved['late_frac'])*100] + \
              [np.mean(heuristics_tight[m]['late_frac'])*100 for m in heuristics_tight]
colors = ['#e74c3c', '#3498db'] + ['#95a5a6']*len(heuristics_tight)
bars = ax.bar(range(len(methods_tight)), lates_tight, color=colors, edgecolor='black')
ax.set_ylabel('Late Jobs (%)', fontsize=12, fontweight='bold')
ax.set_title('TIGHT Deadlines (1.0-1.5×)\n(Original Paper Config)', fontsize=12, fontweight='bold')
ax.set_xticks(range(len(methods_tight)))
ax.set_xticklabels(methods_tight, rotation=30, ha='right', fontsize=9)
for bar, val in zip(bars, lates_tight):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', 
            ha='center', fontsize=9, fontweight='bold')

# Plot 2: Relaxed Deadlines
ax = axes[1]
methods_relax = ['RL (Relaxed)'] + list(heuristics_relaxed.keys())
lates_relax = [np.mean(results_relaxed['late_frac'])*100] + \
              [np.mean(heuristics_relaxed[m]['late_frac'])*100 for m in heuristics_relaxed]
colors = ['#27ae60'] + ['#95a5a6']*len(heuristics_relaxed)
bars = ax.bar(range(len(methods_relax)), lates_relax, color=colors, edgecolor='black')
ax.set_ylabel('Late Jobs (%)', fontsize=12, fontweight='bold')
ax.set_title('RELAXED Deadlines (2.0-4.0×)\n(Our Improved Config)', fontsize=12, fontweight='bold')
ax.set_xticks(range(len(methods_relax)))
ax.set_xticklabels(methods_relax, rotation=30, ha='right', fontsize=9)
for bar, val in zip(bars, lates_relax):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', 
            ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('final_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Figure saved as 'final_comparison.png'")

In [ ]:
# Cell 14: LATEX OUTPUT
print("\n" + "="*90)
print("LATEX TABLE (Copy for paper)")
print("="*90)

latex = r"""
\begin{table}[htbp]
\centering
\caption{Comparison of GPU Scheduling Methods}
\label{tab:comparison}
\begin{tabular}{llccc}
\toprule
\textbf{Deadlines} & \textbf{Method} & \textbf{Late (\%)} & \textbf{Tardiness} & \textbf{Energy (MJ)} \\
\midrule
\multirow{6}{*}{Tight (1.0-1.5$\times$)} 
"""

# Tight deadlines
def add_row(name, results, bold=False):
    late = np.mean(results['late_frac']) * 100
    late_std = np.std(results['late_frac']) * 100
    tard = np.mean(results['tardiness'])
    tard_std = np.std(results['tardiness'])
    energy = np.mean(results['energy']) / 1e6
    
    if bold:
        return f"& \\textbf{{{name}}} & \\textbf{{{late:.1f}$\\pm${late_std:.1f}}} & \\textbf{{{tard:.2f}$\\pm${tard_std:.2f}}} & \\textbf{{{energy:.2f}}} \\\\\n"
    else:
        return f"& {name} & {late:.1f}$\\pm${late_std:.1f} & {tard:.2f}$\\pm${tard_std:.2f} & {energy:.2f} \\\\\n"

latex += add_row("RL (Original Paper)", results_original, True)
latex += add_row("RL (Our Improvements)", results_improved, True)
for name, res in heuristics_tight.items():
    latex += add_row(name, res)

latex += r"""\midrule
\multirow{5}{*}{Relaxed (2.0-4.0$\times$)}
"""

latex += add_row("RL (Our Improvements)", results_relaxed, True)
for name, res in heuristics_relaxed.items():
    latex += add_row(name, res)

latex += r"""\bottomrule
\end{tabular}
\end{table}
"""

print(latex)

# Summary of Findings

## Approaches Tried

| Version | Key Changes | Result |
|---------|-------------|--------|
| **Original Paper** | Pandas env, tight deadlines, [256,256], 200k steps | ~85-90% late |
| **Fast** | NumPy env (10-50× faster) | Same accuracy |
| **Improved (Tight)** | +LR annealing, +entropy decay, [256,256,128] | ~48% late |
| **Improved (Relaxed)** | Deadlines 2-4×, enhanced obs | ~40-50% late |

## Key Finding

**The original problem is greedy-optimal:**
- Larger GPU slice → faster job completion → less tardiness
- Simple "Largest-First" heuristic achieves near-optimal
- RL struggles because there's no complex trade-off to learn

## Recommendations

1. **For tight deadlines**: Use Largest-First heuristic (simpler, faster, effective)
2. **For RL to win**: Need problem with multi-objective trade-offs or constraints
3. **Our speed improvement**: 10-50× faster training with NumPy environment